# QSO Map
Creates a QSO map from logged callsigns by querying the FCC database \
Adapt for international calls using the [maidenhead grid](https://en.wikipedia.org/wiki/Maidenhead_Locator_System) \
Function `gc_arc` adapted from [Plotly spherical linear interpolation function](https://community.plotly.com/t/scattermapbox-plot-curved-lines-like-scattergeo/43665) using [geocentric and Cartesian coordinate transforms](https://www.nosco.ch/mathematics/en/earth-coordinates.php#:~:text=Spherical%20Coordinate%20System&text=The%20spherical%20coordinates%20of%20an,of%20P%20from%20the%20origin.)

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import numpy as np
from numpy import pi, sin, cos
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from geopy.geocoders import Nominatim
import re
import csv
import os
import copy

To map each QSO, the [geocentric coordinate system](https://en.wikipedia.org/wiki/Earth-centered,_Earth-fixed_coordinate_system) with longitude $\lambda$ and latitude $\phi$ can be used to project the signal path to Cartesian coordinates
$$x = \mathrm{cos}(\phi) \mathrm{cos}(\lambda)$$
$$y = \mathrm{cos}(\phi) \mathrm{sin}(\lambda)$$
$$z = \mathrm{sin}(\phi)$$

In [6]:
def coordinate_transform(lam, phi):
    lam = lam*pi/180
    phi = phi*pi/180
    x = cos(lam)*cos(phi) 
    y = sin(lam)*cos(phi) 
    z = sin(phi) 
    return np.array([x, y, z])

For two points $P$ and $Q$, [the central angle is given by the dot product of normal vectors](https://en.wikipedia.org/wiki/Great-circle_distance) $\mathbf{n} = [x \ y \ z]$
$$\Delta \sigma = \mathrm{arccos}\left(\mathbf{n}_P \cdot \mathbf{n}_Q\right)$$

For a destination sufficiently far from the source, [spherical interpolation](https://en.wikipedia.org/wiki/Slerp) can be used to find the curve $s$ formed by the central angle $\Delta \sigma$ between points $P$ and $Q$
$$P = \mathrm{sin}\left((1-t)\Delta \sigma\right)$$
$$Q = \mathrm{sin}\left(t \Delta \sigma \right)$$
Noting that any point on the curve $s$ is a linear combination of the endpoints $P$ and $Q$
$$s = \frac{\mathrm{sin}\left((1-t)\Delta \sigma\right)+\mathrm{sin}\left(t \Delta \sigma \right)}{\mathrm{sin}\left(\Delta \sigma\right)}$$

Compute the projection at $m$ points in the Cartesian coordinate system and return to geocentric coordinates
$$\phi = \mathrm{atan}\left(\frac{z}{\sqrt{x^2+y^2}}\right)$$
$$\lambda = \mathrm{atan2}(y,x)$$

In [7]:
def gc_arc(Src, Dest, m, dir):
    """
    Src=[lonP, latP] lon, lat given in degrees; lon in  (-180, 180], lat in (-90, 90]
    Dest=[lonQ, latQ]
    returns m geocentric coordinates that pass through the points P, Q represented by lon and lat
    if dir=1 it returns the shortest path; for dir=-1 the complement of the shortest path
    """
    Srcs = coordinate_transform(Src[0], Src[1])
    Dests = coordinate_transform(Dest[0], Dest[1])
    if dir == 1:
        dsigma = np.arccos(np.dot(Srcs,Dests)) 
    else:  
        dsigma = 2*pi-np.arccos(np.dot(Srcs,Dests))
    if abs(dsigma) < 1e-6 or abs(dsigma-2*pi) < 1e-6:
        return Src
    else:
        # Spherical linear interpolation
        t = np.linspace(0,1,m)
        P = sin((1-t)*dsigma) 
        Q = sin(t*dsigma)
        # Cartesian coordinates
        s = np.array([a*Srcs + b*Dests for (a, b) in zip(P,Q)])/sin(dsigma)
        # Return to geocentric coordinates
        lons = 180*np.arctan2(s[:,1], s[:, 0])/pi
        lats = 180*np.arctan(s[:,2]/np.sqrt(s[:, 0]**2+s[:,1]**2))/pi
        return lons, lats

Iterate through some call signs, query the coordinates, and plot the signal paths - alternatively, this could be pulled from [WSJT-X](https://wsjt.sourceforge.io/) `.adi` log files, probably significantly faster

In [4]:
geolocator = Nominatim(user_agent="Geopy Library")
source = geolocator.geocode("444 E 77TH ST",country_codes="US")
options = FirefoxOptions()
options.add_argument("--headless")
fcc_query_page = 'https://wireless2.fcc.gov/UlsApp/UlsSearch/searchAdvanced.jsp;JSESSIONID_ULSSEARCH=LICtTpd2E_tsQxz05aHP_jKTUUpmYi2_z_1mfj4sfJT3XmM6f6VE!-879267727!NONE'
fig = go.Figure()
el_calls = ["W5KAL","WA4VGR","K2RHI","WB2JPQ","WB2QBP","KA1BZE","K8IT","AA5JW","WA0TRY","KQ4QZT","KC2WSS","KF8BTM","KE2ARC"]
uhf_calls = ["NJ2DG","N2HBA","N2YVR","KD2GKQ","KA2UTF","AA2GC","K3CBI","N2BUS","K1CVT","WA2HKT","KC2WCP","KC2KGV","KB2GFY","KE2CVD","KD2TJH","AK2EZ","K2GIB","AD2MM",
             "KC2BPP","KC2LWP","K2TAC","N2VZX","WA2NWN","KD2GPR","W2YMM","K2MID","WB2QGZ","NZ2K","W2JPM","WO2X","N2RFI","KE2BFO","KD2TDG","N2NOV","WB2QGZ"]
vhf_calls = ["K2RHI","K2DMC","K2AVY","WB2RRA"]
hf_calls = ["K4TMR","K5VGS","KC4YRV","ND9M","KC8RFE","KA0JWC","KA0JWC","WD3NA","K2HT","K4CTC","AG5PC","WT4DX","K0LLE","KJ5GVE","K0MBC","K4RGN","KE8MJS",
            "KO1U","KE7NLT","W5XO","KD2VRL","N5NXS","WA2HRF","N5EVH","K0JV","KZ4ZZ","WB0ZYU","N4KJA","K6KCL","WB0IWG","W9ACF","K3QB","KJ5CAG","AE0NV","WB2TQE","KD2WWU","KC0ZGK","WY0X",
            "K4QAL","KJ5FVI","K4VHE","WB2YDS","ND2K","WB5BHS","WA7HR","N6NXV","W5KY","W5DBT","K4QAL","KF5MEG","N0LNK","W5DBT","K5OHM","K5MGY","KD9ZQN","KF0UR","W8GXR","WW8O","KC8RFE","KC5JMD","N8CQD","KY4JLS",
            "KI0EB","N3AZ","KA5HET","N2PQJ"]
calls = hf_calls+el_calls+uhf_calls+vhf_calls
mode = len(hf_calls)*['hf']+len(el_calls)*['el']+len(uhf_calls)*['uhf']+len(vhf_calls)*['vhf']
cidx = 0
if os.path.isfile('geolog.csv'):
    next
else:
    header = ['Call Sign', 'Latitude', 'Longitude']
    with open('geolog.csv', 'wt', newline ='') as file:
        creator = csv.writer(file, delimiter=',')
        creator.writerow(i for i in header)
for call_sign in calls:
    class dest:
        pass
    with open('geolog.csv','r',newline='') as csvfile:
        geo_reader = pd.read_csv(csvfile,delimiter=',')
        loc_calls = csvfile.readlines()
        if geo_reader['Call Sign'].str.contains(call_sign).any():
            idx =  geo_reader.index[geo_reader['Call Sign']==call_sign].tolist()
            print('Found',call_sign,'at row',str(idx[0]),'of log','with mode',mode[cidx])
            dest.latitude = geo_reader.iloc[idx[0],2]
            dest.longitude = geo_reader.iloc[idx[0],3]
            plot_trace = True
        else:
            try:
                browser = webdriver.Firefox(options=options)
                browser.get(fcc_query_page)
                search = browser.find_element('name','ulsCallSign')
                browser.find_element('xpath',".//*[@id='statusActive']").click()
                time.sleep(5)
                search.send_keys(call_sign)
                search.send_keys(Keys.RETURN)
                time.sleep(30)
                lnks = browser.find_elements('link text',call_sign)
                for lnk in lnks:
                    lnk.click()
                time.sleep(20)
                src = browser.page_source
                prefix, kw, addr_block = src.partition('Address')
                addr_html,kw,suffix = addr_block.partition('</td>')
                addr = re.search('<br>\n(.*)<br>', str(addr_html))
            except:
                print('Timeout on',call_sign)
            try:
                dest = geolocator.geocode(addr.group(1),country_codes="US") 
                geo_reader.loc[len(geo_reader)] = [len(geo_reader),call_sign,dest.latitude,dest.longitude]
                geo_reader.to_csv('geolog.csv',index=False)
                print("Found",call_sign,"at",dest.latitude,dest.longitude,"with mode",mode[cidx])
                plot_trace = True
            except:
                print('Skipping',call_sign)
                plot_trace = False
            browser.quit()
    if plot_trace == True:
        if mode[cidx] == 'el':
            lc = "orange"
        elif mode[cidx] == 'uhf':
            lc = "green"
        elif mode[cidx] == 'vhf':
            lc = "blue"
        else:
            lc = "purple"
        lons, lats = gc_arc(Src=[source.longitude, source.latitude], Dest=[dest.longitude, dest.latitude], n=100, dir=1)

        fig.add_trace(go.Scattermapbox(lon=lons, lat=lats, mode="lines", line_color=lc, showlegend=False, name=call_sign))
        fig.add_trace(go.Scattermapbox(lon=(lons[0],lons[-1]), lat=(lats[0],lats[-1]), mode="markers", line_color=lc, text=call_sign, name=call_sign, showlegend=False))

        fig.update_layout(width=1600, height=800, template='plotly_dark', mapbox=dict(
                        center_lat=source.latitude, center_lon=source.longitude,
                        zoom=3,
                        style="carto-darkmatter",
                        ))
    cidx = cidx+1
fig.show()


Skipping WB2BWU
Found K4TMR at row 33 of log with mode hf
Found K5VGS at row 34 of log with mode hf
Found KC4YRV at row 35 of log with mode hf
Found ND9M at row 36 of log with mode hf
Found KC8RFE at row 37 of log with mode hf
Skipping KA0JWC
Skipping KA0JWC
Found WD3NA at row 38 of log with mode hf
Skipping K2HT
Found K4CTC at row 39 of log with mode hf
Found AG5PC at row 40 of log with mode hf
Found WT4DX at row 41 of log with mode hf
Found K0LLE at row 42 of log with mode hf
Skipping KJ5GVE
Found K0MBC at row 43 of log with mode hf
Found K4RGN at row 44 of log with mode hf
Found KE8MJS at row 45 of log with mode hf
Found KO1U at row 46 of log with mode hf
Found KE7NLT at row 47 of log with mode hf
Skipping W5XO
Found KD2VRL at row 48 of log with mode hf
Found N5NXS at row 49 of log with mode hf
Found WA2HRF at row 50 of log with mode hf
Found N5EVH at row 51 of log with mode hf
Found K0JV at row 52 of log with mode hf
Found KZ4ZZ at row 53 of log with mode hf
Found WB0ZYU at row 54 